# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [19]:
import os
import pandas as pd
import numpy as np

In [45]:
# Carregar o arquivo CSV
csv_file_path = os.path.join('..', 'demo01.csv')
df = pd.read_csv(csv_file_path)

In [47]:
# Exibir as primeiras linhas do DataFrame
print("Primeiras linhas do DataFrame:")
print(df.head())

Primeiras linhas do DataFrame:
  sexo posse_de_veiculo posse_de_imovel  qtd_filhos            tipo_renda  \
0    M                Y               Y           0               Working   
1    F                N               Y           0  Commercial associate   
2    F                N               Y           0  Commercial associate   
3    M                Y               Y           0               Working   
4    F                Y               N           0               Working   

                        educacao          estado_civil    tipo_residencia  \
0  Secondary / secondary special               Married  House / apartment   
1  Secondary / secondary special  Single / not married  House / apartment   
2  Secondary / secondary special  Single / not married  House / apartment   
3               Higher education               Married  House / apartment   
4              Incomplete higher               Married  House / apartment   

       idade  tempo_emprego  possui_celular

In [49]:
# Tipos de dados de cada variável
print("\nTipos de dados de cada variável:")
print(df.dtypes)


Tipos de dados de cada variável:
sexo                      object
posse_de_veiculo          object
posse_de_imovel           object
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                    float64
tempo_emprego            float64
possui_celular             int64
possui_fone_comercial      int64
possui_fone                int64
possui_email               int64
qt_pessoas_residencia    float64
mau                         bool
dtype: object


In [51]:
# Quantidade de valores ausentes (missings)
print("\nQuantidade de valores ausentes (missings) por variável:")
print(df.isnull().sum())


Quantidade de valores ausentes (missings) por variável:
sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
possui_celular           0
possui_fone_comercial    0
possui_fone              0
possui_email             0
qt_pessoas_residencia    0
mau                      0
dtype: int64


In [53]:
# Distribuição da variável resposta (mau)
if 'mau' in df.columns:
    print("\nDistribuição da variável resposta (mau):")
    print(df['mau'].value_counts())
    print("\nDistribuição percentual da variável resposta (mau):")
    print(df['mau'].value_counts(normalize=True) * 100)
else:
    print("\nA variável 'mau' não está presente no DataFrame.")


Distribuição da variável resposta (mau):
mau
False    16260
True       390
Name: count, dtype: int64

Distribuição percentual da variável resposta (mau):
mau
False    97.657658
True      2.342342
Name: proportion, dtype: float64


#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [55]:
# Criar um DataFrame com os nomes das variáveis e seus tipos de dados
variables_info = pd.DataFrame({
    'Variable': df.columns,
    'Data Type': df.dtypes.astype(str)
})

# Exibir o DataFrame
print(variables_info)

                                    Variable Data Type
sexo                                    sexo    object
posse_de_veiculo            posse_de_veiculo    object
posse_de_imovel              posse_de_imovel    object
qtd_filhos                        qtd_filhos     int64
tipo_renda                        tipo_renda    object
educacao                            educacao    object
estado_civil                    estado_civil    object
tipo_residencia              tipo_residencia    object
idade                                  idade   float64
tempo_emprego                  tempo_emprego   float64
possui_celular                possui_celular     int64
possui_fone_comercial  possui_fone_comercial     int64
possui_fone                      possui_fone     int64
possui_email                    possui_email     int64
qt_pessoas_residencia  qt_pessoas_residencia   float64
mau                                      mau      bool


In [57]:
# Inicializar uma lista vazia para armazenar a quantidade de categorias
qtd_categorias = []

# Percorrer os nomes das variáveis e contar o número de categorias (valores únicos)
for col in df.columns:
    qtd_categorias.append(df[col].nunique())

# Adicionar a nova coluna ao DataFrame
variables_info['qtd_categorias'] = qtd_categorias

# Exibir o DataFrame
print(variables_info)

                                    Variable Data Type  qtd_categorias
sexo                                    sexo    object               2
posse_de_veiculo            posse_de_veiculo    object               2
posse_de_imovel              posse_de_imovel    object               2
qtd_filhos                        qtd_filhos     int64               8
tipo_renda                        tipo_renda    object               5
educacao                            educacao    object               5
estado_civil                    estado_civil    object               5
tipo_residencia              tipo_residencia    object               6
idade                                  idade   float64            5298
tempo_emprego                  tempo_emprego   float64            3005
possui_celular                possui_celular     int64               1
possui_fone_comercial  possui_fone_comercial     int64               2
possui_fone                      possui_fone     int64               2
possui

#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [59]:
# Identificar variáveis qualitativas que precisam ser convertidas em variáveis dummy
qualitative_vars = df.select_dtypes(include=['object', 'category']).columns

# Criar variáveis dummy para essas variáveis
df_dummies = pd.get_dummies(df, columns=qualitative_vars, drop_first=True)

# Mostrar as primeiras linhas do DataFrame com variáveis dummy
print(df_dummies.head())

# Verificar os tipos de dados das variáveis restantes para garantir que não há mais variáveis do tipo str
print(df_dummies.dtypes)

   qtd_filhos      idade  tempo_emprego  possui_celular  \
0           0  58.832877       3.106849               1   
1           0  52.356164       8.358904               1   
2           0  52.356164       8.358904               1   
3           0  46.224658       2.106849               1   
4           0  29.230137       3.021918               1   

   possui_fone_comercial  possui_fone  possui_email  qt_pessoas_residencia  \
0                      0            0             0                    2.0   
1                      0            1             1                    1.0   
2                      0            1             1                    1.0   
3                      1            1             1                    2.0   
4                      0            0             0                    2.0   

     mau  sexo_M  ...  educacao_Secondary / secondary special  \
0  False    True  ...                                    True   
1  False   False  ...                         

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [61]:
# Tabela cruzada entre 'mau' e 'possui_email'
crosstab_email = pd.crosstab(df['mau'], df['possui_email'], normalize='columns')

# Tabela cruzada entre 'mau' e 'posse_de_veiculo'
crosstab_veiculo = pd.crosstab(df['mau'], df['posse_de_veiculo'], normalize='columns')

# Exibir as tabelas cruzadas
print("Tabela cruzada entre 'mau' e 'possui_email':")
print(crosstab_email)

print("\nTabela cruzada entre 'mau' e 'posse_de_veiculo':")
print(crosstab_veiculo)

Tabela cruzada entre 'mau' e 'possui_email':
possui_email         0        1
mau                            
False         0.976269  0.97973
True          0.023731  0.02027

Tabela cruzada entre 'mau' e 'posse_de_veiculo':
posse_de_veiculo         N         Y
mau                                 
False             0.975732  0.977905
True              0.024268  0.022095


In [ ]:
# A diferença na probabilidade de mau = 1 entre as categorias de possui_email (0.003461) é um pouco maior do que a diferença entre as categorias 
# de posse_de_veiculo (0.002173). Portanto a variável possui_email parece ser um pouco mais poderosa para prever a probabilidade de  mau = 1 em 
# comparação com posse_de_veiculo.

#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [63]:
df_dummies.to_csv('demo01_modificado.csv', index=False)